In [1]:
import os
import sys
import pandas as pd

In [2]:
proj_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
proj_path

'C:\\Users\\kweni\\PycharmProjects\\bus_wmata'

In [3]:
# add to python path for relative import of project modules
module_path = os.path.abspath(os.path.join(proj_path, '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join(proj_path))
if module_path not in sys.path:
    sys.path.append(module_path)

## Create Nodes Sheet

In [21]:
stops_csv = os.path.join(
    proj_path, 'data', 'stops', '2021', '08', 'stops_08-18-2021_21-48-43.csv'
)
stops_df = pd.read_csv(stops_csv)
stops_df

,StopID,Name,Lat,Lon,Routes
0,1002585,WISCONSIN AVE NW + FESSENDEN ST NW,38.954858,-77.082833,"['30N', '30S', '31', '33']"
1,1002586,RIGGS RD NE + N CAPITOL ST NE,38.954876,-77.008307,"['64', 'E4', 'E4/', 'K2', 'K6', 'K6*2']"
2,1002587,MISSOURI AVE NW + NEW HAMPSHIRE AVE NW,38.954951,-77.009811,"['E4', 'E4/']"
3,1002588,1ST PL NE + RIGGS RD NE,38.955121,-77.004534,"['64', 'E4', 'E4*1', 'K2', 'K6', 'R1', 'R2']"
4,1002589,MISSOURI AVE NW + NEW HAMPSHIRE AVE NW,38.955145,-77.009968,"['E4', 'E4*1']"
...,...,...,...,...,...
9043,3003055,TRADE ZONE AVE + #16155,38.880427,-76.725650,['C21']
9044,3003056,TRADE ZONE AVE + PRINCE GEORGES BLVD,38.880260,-76.723039,"['C21', 'C22']"
9045,3003057,PRINCE GEORGES BLVD + TRADE ZONE AVE,38.879156,-76.722878,['C21']
9046,3003058,PRINCE GEORGES BLVD + BRANCH CT,38.876736,-76.724502,['C21']


#### CLEAN StopID
As you can see below, there are 7 stops with ID = 0. For network analysis we need unique ID values for each stop node, so can't use the StopId column for network IDs; will need to mask the 0s into unique values.

###### Solution:
As you can see below from 10 smallest output, apart from the 0s, StopIDs are 7 digit long. So will convert the 0s, into unique two digits value of 10s as their respective Id column. For the remaining, Id = StopID.

In [5]:
stops_df.StopID.value_counts()

0          7
1002585    1
3001461    1
3001455    1
3001456    1
          ..
3004021    1
3004020    1
3004016    1
1003309    1
3003059    1
Name: StopID, Length: 9042, dtype: int64

In [13]:
stops_df.nsmallest(10, 'StopID')

,StopID,Name,Lat,Lon,Routes
8684,0,BUS COLLECTION POINT I/B,38.896931,-77.071563,"['38B', '4B']"
8685,0,ANNOUNCEMENT STOP + WEST END OF HOV - RTE 236,38.816921,-77.138084,"['17B', '17M', '29G']"
8867,0,MARK CENTER OUTBOUND STOP,38.832134,-77.116234,"['7A', '7A*3', '7F', '7F*2', '7M']"
8868,0,NAYLOR RD STA OUTBOUND,38.849442,-76.956474,['H12']
8869,0,PENTAGON ANNOUNCEMENT,38.866695,-77.053047,"['10A', '16A', '16C', '16E', '17B', '17G', '17..."
8870,0,SSTC LL COLLECTION POINT I/B,38.994056,-77.030829,"['J1', 'J1*2', 'J2', 'J2*2', 'S2', 'S2*1', 'S9']"
8871,0,SSTC ML COLLECTION POINT I/B,38.993831,-77.029731,"['70', '79', '79*1', 'F4', 'Q1', 'Q2', 'Q4', '..."
384,1000001,SHEPHERD PKY SW + SHEPHERD PKY SW,38.816082,-77.017741,"['A4', 'A4*3', 'W5']"
385,1000003,SHEPHERD PKY SW + DC VILLAGE LN,38.816293,-77.018108,"['A4', 'A4*1', 'W5']"
386,1000004,MARTIN LUTHER KING JR AVE SW + JOLIET ST SW,38.819634,-77.010820,"['A4', 'A4*1', 'A4*3', 'A4*4', 'W1']"


In [22]:
# add Id column as first column and transform 0s
stops_df.insert(0, 'Id', stops_df.StopID)
for n, idx in enumerate(stops_df[stops_df.StopID == 0].index, 1):
    node_id = 10 * n
    stops_df.loc[idx, 'Id'] = node_id
stops_df.nsmallest(10, 'StopID')

,Id,StopID,Name,Lat,Lon,Routes
8684,10,0,BUS COLLECTION POINT I/B,38.896931,-77.071563,"['38B', '4B']"
8685,20,0,ANNOUNCEMENT STOP + WEST END OF HOV - RTE 236,38.816921,-77.138084,"['17B', '17M', '29G']"
8867,30,0,MARK CENTER OUTBOUND STOP,38.832134,-77.116234,"['7A', '7A*3', '7F', '7F*2', '7M']"
8868,40,0,NAYLOR RD STA OUTBOUND,38.849442,-76.956474,['H12']
8869,50,0,PENTAGON ANNOUNCEMENT,38.866695,-77.053047,"['10A', '16A', '16C', '16E', '17B', '17G', '17..."
8870,60,0,SSTC LL COLLECTION POINT I/B,38.994056,-77.030829,"['J1', 'J1*2', 'J2', 'J2*2', 'S2', 'S2*1', 'S9']"
8871,70,0,SSTC ML COLLECTION POINT I/B,38.993831,-77.029731,"['70', '79', '79*1', 'F4', 'Q1', 'Q2', 'Q4', '..."
384,1000001,1000001,SHEPHERD PKY SW + SHEPHERD PKY SW,38.816082,-77.017741,"['A4', 'A4*3', 'W5']"
385,1000003,1000003,SHEPHERD PKY SW + DC VILLAGE LN,38.816293,-77.018108,"['A4', 'A4*1', 'W5']"
386,1000004,1000004,MARTIN LUTHER KING JR AVE SW + JOLIET ST SW,38.819634,-77.010820,"['A4', 'A4*1', 'A4*3', 'A4*4', 'W1']"


## Create Edges Sheet

In [36]:
paths_csv = os.path.join(
    proj_path, 'data', 'path_details_stops', '2021', '08', '19',
    'path_details_stops_3Av1_08-19-2021_22-43-32.csv'
)
paths_df = pd.read_csv(paths_csv)
paths_df.head()

,RouteName,RouteID,DirectionNum,DirectionText,TripHeadsign,Lat,Lon,Name,Routes,StopID,StopNum
0,3A - 7 CORNERS - ANNANDALE,3Av1,0,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,38.825707,-77.219313,PATRIOT DR + AMERICANA DR,"['16A', '29G', '3Av1']",5000689,1
1,3A - 7 CORNERS - ANNANDALE,3Av1,0,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,38.825832,-77.217033,PATRIOT DR + #7923,"['16A', '29G', '3Av1']",5000693,2
2,3A - 7 CORNERS - ANNANDALE,3Av1,0,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,38.827706,-77.215689,PATRIOT DR + BRIARWOOD CT N,"['16A', '29G', '3Av1']",5000730,3
3,3A - 7 CORNERS - ANNANDALE,3Av1,0,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,38.828548,-77.213092,PATRIOT DR + FOREST GLEN CT,"['16A', '29G', '3Av1']",5000747,4
4,3A - 7 CORNERS - ANNANDALE,3Av1,0,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,38.829839,-77.210365,HERITAGE DR + MCWHORTER PL,"['16A', '29G', '3Av1']",5000759,5


In [59]:
def get_node_id(df: pd.DataFrame, lat: float, lon: float) -> int:
    """Get Id from nodes dataframe given lat and lon."""
    return df.loc[(df.Lat == lat) & (df.Lon == lon), 'Id'].array
get_node_id(df=stops_df, lat=38.993831, lon=-77.029731)

<PandasArray>
[70]
Length: 1, dtype: int64

In [64]:
# expected edges dataframe structure
# edges_df = pd.DataFrame(
#     columns=[
#         'Source', 'Target', 'Type', 'Weight',
#         'RouteID', 'DirectionText', 'TripHeadSign', 'PathSeq'
#     ]
# )

In [69]:
edges = list()
# iterate rows in paths df to generate edges data
for i in range(1, len(paths_df)):
    cur_dir, cur_stopID, cur_lat, cur_lon = paths_df.loc[
        i, ['DirectionText', 'StopID', 'Lat', 'Lon']
    ]
    source_row = paths_df.iloc[i - 1]
    
    # skip to next row, if current and previous rows are diff paths
    if cur_dir != source_row.DirectionText:
        continue
    
    # remap stopIDs with 0s to appropriate ID node
    if source_row.StopID == 0:
        source = get_node_id(
            df=stops_df, lat=source_row.Lat, lon=source_row.Lon
        )[0]
    else:
        source = source_row.StopID
    
    if cur_stopID == 0:
        target = get_node_id(
            df=stops_df, lat=cur_lat, lon=cur_lon
        )[0]
    else:
        target = cur_stopID
    
    # define edge node for path
    edges.append({
        'Source': source, 'Target': target, 'Type': 'Directed', 'Weight': 1,
        'RouteId': source_row.RouteID,
        'DirectionText': source_row.DirectionText,
        'TripHeadSign': source_row.TripHeadsign,
        'PathSeq': source_row.StopNum
    })
edges_df = pd.DataFrame.from_records(edges)
edges_df.head()

,Source,Target,Type,Weight,RouteId,DirectionText,TripHeadSign,PathSeq
0,5000689,5000693,Directed,1,3Av1,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,1
1,5000693,5000730,Directed,1,3Av1,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,2
2,5000730,5000747,Directed,1,3Av1,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,3
3,5000747,5000759,Directed,1,3Av1,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,4
4,5000759,5000790,Directed,1,3Av1,EAST,SEVEN CORNERS VIA EAST FALLS CHURCH STATION,5
